In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 =pd.read_csv('train.csv')
df_2 =pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
df_1.head()
df_2.head()

In [ ]:
df = pd.concat([
    df_1,df_2
],)

df.head()

In [ ]:
df.head()

In [ ]:
df.isna()
df.isna().sum().sort_values(ascending = False)
num_cols = df._get_numeric_data().columns

In [ ]:
df.isna().sum().sort_values(ascending = False)

In [3]:
num_cols = df._get_numeric_data().columns
num_set = df[num_cols]

NameError: name 'df' is not defined

In [4]:
num_set.head()

NameError: name 'num_set' is not defined

In [5]:
num_set.describe(include='all')

NameError: name 'num_set' is not defined

In [6]:
corr = num_set.corr()
corr.style.background_gradient(cmap='coolwarm')

NameError: name 'num_set' is not defined

In [7]:
df.head()

NameError: name 'df' is not defined

In [8]:
## Clean Variables

df[['LotFrontage','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtHalfBath','BsmtFullBath']] = df[['LotFrontage', 'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']].fillna(0)
df [['Alley', 'BsmtQual', 'MasVnrType','MasVnrArea','GarageYrBlt','BsmtCond',  'BsmtExposure',  'BsmtFinType1','BsmtFinType2', 'FireplaceQu', 'GarageType',  'GarageFinish',  'GarageQual',  'GarageCond',  'PoolQC',  'Fence',  'MiscFeature']] = df[['Alley',  'BsmtQual','GarageYrBlt', 'BsmtCond',  'BsmtExposure',  'BsmtFinType1', 'MasVnrType','MasVnrArea','BsmtFinType2',  'FireplaceQu',  'GarageType',  'GarageFinish',  'GarageQual',  'GarageCond',  'PoolQC',  'Fence', 'MiscFeature']] .fillna('None')
df.drop = df[['MasVnrType', 'MasVnrArea', 'Electrical', 'GarageYrBlt']]




NameError: name 'df' is not defined

In [9]:
df.isna().sum().sort_values(ascending = False)

NameError: name 'df' is not defined